# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-30 02:05:18] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-30 02:05:18] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-30 02:05:18] INFO utils.py:164: NumExpr defaulting to 16 threads.


<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.nvrtc module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.nvrtc module instead.


[2026-01-30 02:05:21] INFO server_args.py:1767: Attention backend not specified. Use fa3 backend by default.


...

Capturing batches (bs=128 avail_mem=76.93 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=88 avail_mem=76.21 GB):  15%|█▌        | 3/20 [00:01<00:04,  3.84it/s]

Capturing batches (bs=56 avail_mem=76.12 GB):  30%|███       | 6/20 [00:01<00:01,  8.22it/s]

Capturing batches (bs=16 avail_mem=76.10 GB):  65%|██████▌   | 13/20 [00:01<00:00, 14.65it/s]

Capturing batches (bs=1 avail_mem=76.08 GB): 100%|██████████| 20/20 [00:01<00:00, 10.81it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Tanya and I'm a software engineer. My goal is to help people use the internet more efficiently. As someone who has grown up in a world where the internet was largely a novelty, I've had to learn how to navigate it myself. Therefore, my skill set is focused on understanding web protocols, best practices for websites, and building websites from scratch. I'm also proficient in a variety of programming languages including JavaScript and Python.

I'm passionate about helping people find and access information they need, but I also understand that too much information is overwhelming. That's why I like to design websites that are easy to navigate and provide clear,
Prompt: The president of the United States is
Generated text:  a position that has been vacant for a long time, and the candidate that he nominates to fill the vacancy is likely to be a very different kind of person than the previous one. For instance, if the previous president was a cons

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I am a [Age] year old [Gender] [Occupation]. I am a [Occupation] who has always been [Positive Traits] and [Negative Traits]. I am passionate about [My Passion] and I am always [My Passion] to [My Passion]. I am [My Passion] and I am [My Passion]. I am [My Passion] and I am [My Passion]. I am [My Passion] and I am [My Passion]. I am [My Passion] and I am [My Passion]. I am [My Passion] and I am [My Passion]. I am [

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as the City of Light, and is the largest city in Europe by population. It is located on the Seine River and is the seat of government, administration, and culture for the country. Paris is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. The city is also famous for its cuisine, fashion, and art scene. Paris is a cultural and economic hub of Europe and is a popular tourist destination. It is home to many world-renowned museums, theaters, and art galleries. The city is also known for its nightlife, with many bars and

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the way we interact with technology and the world around us. Here are some of the most likely trends that are expected to shape the future of AI:

1. Increased Integration of AI into Everyday Life: As AI becomes more integrated into our daily lives, we are likely to see more and more AI-powered tools and services being used to enhance our lives. This could include things like smart home devices, virtual assistants, and self-driving cars.

2. Greater Use of AI for Predictive Analytics: AI is already being used to make predictions about the future, and it is likely



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [name], and I am a [occupation or profession]. I am a [field of study], and I specialize in [current project or area of interest]. I enjoy [my hobbies or interests]. What brings you to this field? How do you feel about being an [occupation or profession]?

[Name] is a [profession] who is passionate about [career goal or project]. They are always looking for ways to improve themselves and try new things. They are always eager to learn and take risks. They have a great sense of humor and are always looking for new ways to entertain themselves. They are very grateful for their opportunities to

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. The city is known for its historical landmarks, fashion scene, and iconic landmarks such as Notre Dame Cathedral, the Eiffel Tower, and the Louvre Museum. Paris is a b

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 are

 becoming

 a

 reality

,

In [6]:
llm.shutdown()